In [1]:
import nidaqmx
import time
from time import ctime
import numpy as np
import csv
import nidaqmx.system
from rich import print

In [2]:
system = nidaqmx.system.System.local()
devices = system.devices

In [3]:
print("장치를 선택해주세요")
for idx, device in enumerate(devices):
    print(idx, ']', device.name)

selected = int(input())
device = devices[selected]

장치를 선택해주세요

0 ] cDAQ1

1 ] cDAQ1Mod1

1


In [4]:
task = nidaqmx.Task()
print('add input channels')

channels = device.ai_physical_chans
for channel in channels:
    print("channel : ", channel.name)
    task.ai_channels.add_ai_voltage_chan(channel.name)
    
print()
print('add output channels')
channels = device.ao_physical_chans
for channel in channels:
    print("channel : ", channel.name)
    task.ao_channels.add_ao_voltage_chan(channel.name)

task.timing.cfg_samp_clk_timing(rate=51200,
                                active_edge=nidaqmx.constants.Edge.RISING,
                                sample_mode=nidaqmx.constants.AcquisitionType.FINITE,
                                samps_per_chan=51200)
print('task ready')

add input channels

channel :  cDAQ1Mod1/ai0

channel :  cDAQ1Mod1/ai1

channel :  cDAQ1Mod1/ai2

channel :  cDAQ1Mod1/ai3

add output channels

task ready

In [5]:
for t in device.ao_physical_chans:
    print(t)

In [6]:
filePath = input("저장할 파일명을 입력해주세요")
    
writerList = []
for channel in task.channels:
    name = filePath +"_"+ channel.name.replace("/", "_") + ".csv"
    print('name = ', name)
    writerList.append(csv.writer(open(name, 'a', newline='\n')))
    
writerList

저장할 파일명을 입력해주세요dsa


name =  dsa_cDAQ1Mod1_ai0.csv

name =  dsa_cDAQ1Mod1_ai1.csv

name =  dsa_cDAQ1Mod1_ai2.csv

name =  dsa_cDAQ1Mod1_ai3.csv

In [9]:
datas = task.read(number_of_samples_per_channel=100)
for idx, data in enumerate(datas):
    for raw in data:
        writerList[idx].writerow([raw])